In [31]:
import numpy as np
import keras
import pickle as pk
import matplotlib.pyplot as plt
import os
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.layers import UpSampling2D, Input
from keras.models import Model, Sequential

In [57]:
### Define model

# Load ResNet NN and keep only the relevant layers

#TODO: add dropout

#resnet = ResNet50(weights='imagenet', include_top=True)
resnet = ResNet50(weights=None, include_top=True)
resnet.layers.pop()
resnet.outputs = [resnet.layers[-1].output]
resnet.layers[-1].outbound_nodes = []

# Def right and left inputs
left_img = Input(shape=(32, 32, 3), name='left_input')
right_img = Input(shape=(32, 32, 3), name='right_input')
left_inp = UpSampling2D(size =(7,7))(right_img)
right_inp = UpSampling2D(size =(7,7))(left_img)

# Def model
left_out = resnet(left_inp)
right_out = resnet(right_inp)
diff = keras.layers.subtract([left_out, right_out])
prediction = keras.layers.Dense(1,activation='sigmoid', name='dist')(diff)
siamese_net = Model(input=[left_img,right_img],output=prediction)

# Compile model
adadelta = keras.optimizers.Adadelta()
siamese_net.compile(loss='binary_crossentropy', optimizer=adadelta)

# Print information about the model
siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
right_input (InputLayer)        (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
left_input (InputLayer)         (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
up_sampling2d_10 (UpSampling2D) (None, 224, 224, 3)  0           right_input[0][0]                
__________________________________________________________________________________________________
up_sampling2d_11 (UpSampling2D) (None, 224, 224, 3)  0           left_input[0][0]                 
__________________________________________________________________________________________________
resnet50 (

/Users/charles/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("di...)`


### Feature extractor

In [ ]:
# Extract ResNet weight fine tuned
resnet_tuned = siamese_net.layers[4]
img = Input(shape=(32, 32, 3), name='input')
img2 = UpSampling2D(size =(7,7))(img)

# Def model
feat = resnet_tuned(img2)
extractor = Model(input=img, output=feat)
extractor.summary()

### Define custom distance for our k-NN

In [62]:
# Extract distance layer from our model, and use it for the k-NN
dist_layer = siamese_net.get_layer('dist').get_weights()
def cust_dist(x, y):
    """Distance using the metric learned by our SiameseNet
    Args:
        x, y: (array (2048,)): the features to use
    return:
        dist: (float) distance value
    """
    temp = np.abs(x-y)
    return np.dot(dist_layer, temp)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 32, 32, 3)         0         
_________________________________________________________________
up_sampling2d_16 (UpSampling (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


/Users/charles/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("re...)`


In [42]:
def get_pairs(positive_percentage=0.5, cifar_batch_id=1, batch_size=32):
    pairs = []
    feats, labels = load_batch('cifar10', cifar_batch_id)
    indexes = np.random.randint(len(labels),size=batch_size)
    sample_images = np.take(feats, indexes, axis=0)
    sample_labels = np.take(labels, indexes, axis=0)
    other_indexes = np.delete(range(len(labels)), indexes)
    other_feats = np.take(feats, other_indexes, axis=0)
    other_labels = np.take(labels, other_indexes)
    for i in range(batch_size):
        if i < int(batch_size*positive_percentage):
            index_of_label = np.where(other_labels==sample_labels[i])[0]
        else :
            index_of_label = np.where(other_labels!=sample_labels[i])[0]
        index = np.take(index_of_label, np.random.randint(len(index_of_label)))
        other_image = other_feats[index]
        plt.figure()
        plt.subplot("121")
        plt.imshow(other_image)
        plt.subplot("122")
        plt.imshow(sample_images[i])
        pairs.append([sample_images[i], other_image])
    return np.array(pairs)